In [1]:
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('./dataset/clean-sensors.csv', infer_datetime_format=True, header=0, index_col='datetime')

C:\Users\Bintang\AppData\Local\Temp\ipykernel_25492\3182949423.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('./dataset/clean-sensors.csv', infer_datetime_format=True, header=0, index_col='datetime')


In [3]:
data.head()

,Temperature,Humidity,Illumination,kat_temperature,kat_humidity,kat_illumination,Kipas,Humidifier,LED
datetime,,,,,,,,,
03/07/2023 0:00:00,28.0,75,76.61,2,2,3,0,0,1
03/07/2023 0:01:00,28.0,75,75.33,2,2,3,0,0,1
03/07/2023 0:02:00,28.0,75,74.49,2,2,3,0,0,1
03/07/2023 0:03:00,28.0,75,74.49,2,2,3,0,0,1
03/07/2023 0:04:00,28.0,75,74.49,2,2,3,0,0,1


In [4]:
data.columns

Index(['Temperature', 'Humidity', 'Illumination', 'kat_temperature',
       'kat_humidity', 'kat_illumination', 'Kipas', 'Humidifier', 'LED'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11507 entries, 03/07/2023 0:00:00 to 10/07/2023 23:59:00
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature       11507 non-null  float64
 1   Humidity          11507 non-null  int64  
 2   Illumination      11063 non-null  float64
 3   kat_temperature   11507 non-null  int64  
 4   kat_humidity      11507 non-null  int64  
 5   kat_illumination  11507 non-null  int64  
 6   Kipas             11507 non-null  int64  
 7   Humidifier        11507 non-null  int64  
 8   LED               11507 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 899.0+ KB


In [6]:
data = data.drop(['kat_humidity', 'kat_illumination', 'kat_temperature'], axis=1)

In [7]:
stats = data.describe()
stats.pop('Kipas')
stats.pop('LED') 
stats.pop('Humidifier')
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
Temperature,11507.0,30.018954,1.249807,27.20,28.90,30.20,30.80,33.30
Humidity,11507.0,66.055618,5.500173,51.00,62.00,67.00,70.00,76.00
Illumination,11063.0,147.412321,201.545117,28.89,73.24,73.24,76.61,1508.57


In [8]:
train, test = train_test_split(data, test_size=.2, random_state=1)

In [9]:
n_features = len(train.columns)
train.shape

(9205, 6)

In [10]:
def format_output(data):
    fan = np.array(data.pop('Kipas'))
    humid = np.array(data.pop('Humidifier'))
    led = np.array(data.pop('LED'))
    return fan, humid, led    

In [11]:
train_label = format_output(train)
test_label = format_output(test)

In [12]:
train_label

(array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 1, 0, 0], dtype=int64),
 array([0, 1, 0, ..., 1, 1, 0], dtype=int64))

In [13]:
len(train_label[0]) == len(train)

True

In [14]:
inputs = Input(shape=(3, ))
bc = tf.keras.losses.BinaryCrossentropy()
rms = tf.keras.optimizers.RMSprop(learning_rate=.0001)

# Siamese neural network

In [15]:
def base_model(input_layer):
    # x = Conv1D(32, 5, 1, 'causal', activation='relu')(input_layer)
    # x = LSTM(64, return_sequences=True)(input_layer)
    # x = LSTM(64, return_sequences=True)(x)
    x = Dense(128, activation='relu')(input_layer)
    x = Dense(128, activation='relu')(x)
    return x
    

In [16]:
def final_model(input_layer):
    x = base_model(input_layer)
    fan_output = Dense(1, activation='sigmoid', name='fan_output')(x)
    led_output = Dense(1, activation='sigmoid', name='led_output')(x)   
    humid = Dense(1, activation='sigmoid', name='humid_output')(x)
    return tf.keras.Model(inputs=input_layer, outputs=[fan_output, led_output, humid])

In [17]:
model = final_model(inputs)
model.compile(
    loss= {
        'fan_output': bc, 
        'led_output': bc, 
        'humid_output': bc, 
    },
    optimizer=rms,
    metrics={
        'fan_output': 'accuracy', 
        'led_output': 'accuracy', 
        'humid_output': 'accuracy',
    }
)

In [19]:
model.fit(
    x=train,
    y=train_label,
    epochs=25,
    validation_data=(test, test_label)
)

Epoch 1/25
288/288 [==============================] - 4s 14ms/step - loss: nan - fan_output_loss: nan - led_output_loss: nan - humid_output_loss: nan - fan_output_accuracy: 0.9710 - led_output_accuracy: 0.7419 - humid_output_accuracy: 0.2677 - val_loss: nan - val_fan_output_loss: nan - val_led_output_loss: nan - val_humid_output_loss: nan - val_fan_output_accuracy: 0.9683 - val_led_output_accuracy: 0.7242 - val_humid_output_accuracy: 0.2802
Epoch 2/25
288/288 [==============================] - 4s 13ms/step - loss: nan - fan_output_loss: nan - led_output_loss: nan - humid_output_loss: nan - fan_output_accuracy: 0.9710 - led_output_accuracy: 0.7419 - humid_output_accuracy: 0.2677 - val_loss: nan - val_fan_output_loss: nan - val_led_output_loss: nan - val_humid_output_loss: nan - val_fan_output_accuracy: 0.9683 - val_led_output_accuracy: 0.7242 - val_humid_output_accuracy: 0.2802
Epoch 3/25
288/288 [==============================] - 4s 14ms/step - loss: nan - fan_output_loss: nan - led_ou

TypeError: 'Functional' object is not iterable